In [1]:
%load_ext autoreload
%autoreload

This notebook performs preprocessing for all participants in the LDM experiment. Preprocessing consists of: 
* An epoching step which assigns gaze samples to trials 
* An alignment step which aligns gaze samples to AoIs and task cues

In [2]:
import sys
import os

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib as matplotlib
from matplotlib.colors import ListedColormap
import seaborn as sns
import scipy.io as io
import pandas as pd
from pandas import DataFrame, read_csv
from nivlink import Screen, Raw, Epochs, align_to_aoi, compute_fixations, plot_heatmaps
import cv2
import readline
from math import dist
from scipy.spatial.distance import squareform, pdist

import warnings
from scipy.stats import kde
import nivlink
import ipywidgets as wdg
from scipy.stats import iqr
print('Powered by NivLink' + str(nivlink.__version__))

Powered by NivLink0.2.5


## 1. List participants.

In [3]:
all_subjects =  [23, 26, 27, 28, 29, 30, 31, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 49, 53, 54, 
                 55, 57, 58, 60, 61, 62, 63, 66, 67, 68, 69, 70, 71];

print('Found ' + str(len(all_subjects)) + ' subjects.')

Found 38 subjects.


## 2. Helper functions.

To-do eventually: since these are used across multiple notebooks, add to a library. 

In [4]:
def load_subj_data(subj_id):
    
    et_data_dir = os.getcwd().strip('ldm-analysis') + 'RawData/'# this expects RawData to be one directory up from the analysis repo
    edf_path = et_data_dir + 'Sub' + str(subj_id) + 'ET.edf'
    # Read subject's data from edf file.
    data = Raw(edf_path)
    # Filter out only eye position.
    raw_pos_data = data.data[:,0,(0,1)]
    # Grab messages for epoching step.
    messages = data.messages 
 
    return data, raw_pos_data, messages, data.info['sfreq']

In [5]:
def get_run_onsets(messages): 
    """ Returns run onsets. 
        
        This function is specific to how the experiment code handles
        messages to the EDF file during the task.

    Parameters
    ----------
    messages: array, shape (n_times, 1) 
        Array containing messages from the NivLink Raw object.

    Returns
    -------
    run_onsets : array, shape (n_runs, 2)
        Run IDs and start indices.

    """

    n_messages = len(messages)
    print('Found ' + str(n_messages) + ' messages...')
    run_onsets = np.empty((0,3), dtype = int)

    for m in np.arange(n_messages):
        
        this_message_index = messages[m][0]
        this_message = messages[m][1]
        
        ## We encountered a new run.
        if 'Run' in this_message:
             
            ## Get index of first XDAT 2 in this run. 
            ## This is specific to the LDM dataset
            ## because the first stim onset is not align with the Start Run message.   
            first_xdat2_message_index = messages[m+12][0]
            first_xdat2_message = messages[m+12][1]
            
            ## Initialize onset array for this run.
            this_run_onsets = np.empty((1,3), dtype = int); 
            
            this_run_onsets[:,0] = int(this_message.strip('Run '))
            # this_run_onsets[:,1] = int(this_message_index) # actual run start message
            this_run_onsets[:,1] = int(first_xdat2_message_index)  # first XDAT2 in run
              
        ## Re-construct End Run message index by looking at the last trial in each run.
        ## Assumes we consistently had 40 trial per run as per LDM_Run4.m
        if 'Trial 40' in this_message:
            
            last_xdat1_message_index = messages[m+6][0]
            last_xdat1_message = messages[m+6][1]
            
            this_run_onsets[:,2] = int(last_xdat1_message_index)
            run_onsets = np.vstack((run_onsets,this_run_onsets))
            
    return run_onsets

In [6]:
def rereference_events(subj_id, n_blocks, run_onsets, sfreq): 
    """The events file is a `n_trials ` x 3 array which defines: 
    (1) the run/block number 
    (2) the trial onset relative to the first stimulus onset in the run (in seconds) 
    (3) the trial offset (in seconds) 
    Using the sample frequency, it gets re-referenced below to the NivLink 0.2 format: 
    (1) the event onset (in sample indices)
    (2) start time before the event 
    (3) end time after the event """

    # Load file.
    et_data_dir = os.getcwd().strip('ldm-analysis') + 'RawData/'# this expects RawData to be one directory up from the analysis repo
    events_file_path = et_data_dir + str(subj_id) + 'events.mat'
    events_mat = io.loadmat(events_file_path)
    events = np.array(events_mat["events_array"])
    
    # Record run onsets in events array. 
    new_col = events.sum(1)[...,None]*0
    events = np.append(events, new_col, 1)
    for b in np.arange(n_blocks):
        block = b+1
        this_block = events[:,0] == block
        events[this_block,3] = run_onsets[block-1,1].astype(int) 
        
    # Convert to dataframe.
    events_df = pd.DataFrame(events)
    events_df.columns = ['block','1', '2', '3']
    
    # Round events to sampling frequency.   
    events_df['1'] = np.floor(events_df['1'] * sfreq) / sfreq
    events_df['2'] = np.ceil(events_df['2'] * sfreq) / sfreq

    # Re-reference events.
    events_df['3'] += events_df['1'] * sfreq
    events_df['2'] -= events_df['1']
    events_df['1'] -= events_df['1']
    
    return events_df

## 3. Main pre-processing loop.

In [7]:
## Load all centers.
all_centers = pd.read_csv(os.getcwd() + '/allCenters.csv')

In [17]:
## Define screen metadata.
xdim, ydim, n_screens = 1280, 1024, 1 
aoisidelength = 162
n_aois = 9

## Initialize subject level containers.
subj_all = []
percent_null = []

## Loop over participants and preprocess.
for subj_id in all_subjects:
    
    if (subj_id == 58):
        continue

    print("Preprocessing participant ", subj_id)
    subj_all.append(subj_id)

    ## Set save path.
    fixations_path = et_data_dir + str(subj_id) + 'fixations.csv'

    ## Load data.
    data, raw_pos_data, messages, sfreq = load_subj_data(subj_id)

    ## Subset this participant's centers. 
    sub = 'Sub' + str(subj_id) + '_'
    centers = all_centers[all_centers['Unnamed: 0'].str.contains(sub)]
    centers['Block'] = [int(s.replace(sub + 'block' + '_', "")) for s in list(centers['Unnamed: 0'].values)]

    ## Mark run onsets. 
    run_onsets = get_run_onsets(messages)
    n_blocks, d = run_onsets.shape

    ## Make block index indicator of same size as gaze data.
    block_idx = np.zeros(len(raw_pos_data))
    for r in np.arange(n_blocks):  
        block_start = run_onsets[r,1]
        block_end = run_onsets[r,2]    
        block_idx[block_start-1:block_end-1] = r+1

    ## Re-format events dataframe. 
    events_df = rereference_events(subj_id, n_blocks, run_onsets, sfreq)

    ## Load feature map.
    # The feature map file is a `n_trials` x `n_aoi` array which defines the cue corresponding to each AoI.
    et_data_dir = os.getcwd().strip('ldm-analysis') + 'RawData/'# this expects RawData to be one directory up from the analysis repo
    featmap_file_path = et_data_dir + str(subj_id) + 'featmap.mat'
    featmap_mat = io.loadmat(featmap_file_path, struct_as_record=False, squeeze_me=True)
    featmap = np.array(featmap_mat["features_aoi_map"])

    ## Initialize block level containers.
    fixations = []
    total_trials = 0
    
    ## Loop over blocks and preprocess.
    for b in np.arange(n_blocks):

        ## Retrieve block number. 
        block = b+1
        print("Block ", block)

        ## Initialize Screen object.
        print("Adding AoIs... ")
        info = Screen(xdim, ydim, n_screens)

        ## Add AoIs.
        # Grab block centers. 
        block_centers = centers[centers['Block'] == block]
        # Add each AoI based on the custom center. 
        # 1-9 AoI numbering is consistent with left column -> middle column -> right column order.
        for a in np.arange(n_aois):
            col_names = ['aoi' + str(a) + '_x', 'aoi' + str(a) + '_y']
            this_aoi_centers = block_centers[col_names].iloc[0].values
            info.add_rectangle_aoi(this_aoi_centers[0]-aoisidelength//2, this_aoi_centers[0]+aoisidelength//2, this_aoi_centers[1]-aoisidelength//2, this_aoi_centers[1]+aoisidelength//2)

        ## Define null area as its own AoI.
        info.indices[info.indices == 0] = 10

        ## Make epochs.
        print("Epoching... ")
        # Subselect this block's events.
        this_block = events_df.loc[events_df['block'] == block]
        this_block_idx = events_df.index[events_df['block'] == block].values 
        n_trials_block = this_block.shape[0]
        # Make epochs without considering null AoI.
        epochs = Epochs(data, this_block['3'].values, this_block['1'].values, this_block['2'].values, picks='gaze')

        ## Align to AoI.
        print("Aligning to AoI...")
        screen_idx_block = np.zeros((n_trials_block)).astype(int)
        # Subselect featmap.
        this_block_featmap = featmap[this_block_idx,:]
        # Add "null" AoI.
        this_block_featmap = np.hstack((this_block_featmap, np.ones((this_block_featmap.shape[0],1)) * 10))
        n_trials_block = this_block.shape[0]
        # Align. 
        aligned = align_to_aoi(epochs.data[:,:,0:2,:], info, screen_idx_block)

        ## Compute fixations for this block. 
        fixations_block = compute_fixations(aligned[:,0,:], epochs.times)

        ## Map to task cue.
        fixations_block['Feature'] = this_block_featmap[fixations_block.Trial.values.astype(int)-1,fixations_block.AoI.values.astype(int)-1]      

        ## Add trial indicator.
        fixations_block['Trial'] = fixations_block['Trial'] + total_trials
        total_trials = total_trials + n_trials_block 

        ## Append.
        fixations.append(fixations_block)

    ## Concatenate block df. 
    fixations = pd.concat(fixations)

    ## Compute percentage of fixations outside AoI. 
    fixations_after_start = fixations[fixations['Onset'] > 0.2]
    percent_null_subj = np.sum(fixations_after_start[fixations_after_start['AoI'] == 10]['Duration']) / np.sum(fixations_after_start['Duration'])
    print('Percent of time spent looking outside valid AoIs: ')
    print(percent_null_subj)
    percent_null.append(percent_null_subj)

    ## Save.
    fixations.to_csv(fixations_path)

## Save percentage of samples that fall outside AoI.
dist_mean = pd.DataFrame({'subj': subj_all, 
               'percent_null': percent_null})
dist_mean.to_csv('percent_null.csv', index=False)

Preprocessing participant  23


<ipython-input-17-90a5b70bb05f>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  centers['Block'] = [int(s.replace(sub + 'block' + '_', "")) for s in list(centers['Unnamed: 0'].values)]


Found 3493 messages...
Block  1
Adding AoIs... 
Epoching... 
Aligning to AoI...
Block  2
Adding AoIs... 
Epoching... 
Aligning to AoI...
Block  3
Adding AoIs... 
Epoching... 
Aligning to AoI...
Block  4
Adding AoIs... 
Epoching... 
Aligning to AoI...
Block  5
Adding AoIs... 
Epoching... 
Aligning to AoI...
Block  6
Adding AoIs... 
Epoching... 
Aligning to AoI...
Block  7
Adding AoIs... 
Epoching... 
Aligning to AoI...
Block  8
Adding AoIs... 
Epoching... 
Aligning to AoI...
Block  9
Adding AoIs... 
Epoching... 
Aligning to AoI...
Block  10
Adding AoIs... 
Epoching... 
Aligning to AoI...
Percent of time spent looking outside valid AoIs: 
0.1200190819243706
Preprocessing participant  26


<ipython-input-17-90a5b70bb05f>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  centers['Block'] = [int(s.replace(sub + 'block' + '_', "")) for s in list(centers['Unnamed: 0'].values)]


Found 3481 messages...
Block  1
Adding AoIs... 
Epoching... 
Aligning to AoI...
Block  2
Adding AoIs... 
Epoching... 
Aligning to AoI...
Block  3
Adding AoIs... 
Epoching... 
Aligning to AoI...
Block  4
Adding AoIs... 
Epoching... 
Aligning to AoI...
Block  5
Adding AoIs... 
Epoching... 
Aligning to AoI...
Block  6
Adding AoIs... 
Epoching... 
Aligning to AoI...
Block  7
Adding AoIs... 
Epoching... 
Aligning to AoI...
Block  8
Adding AoIs... 
Epoching... 
Aligning to AoI...
Block  9
Adding AoIs... 
Epoching... 
Aligning to AoI...
Block  10
Adding AoIs... 
Epoching... 
Aligning to AoI...
Percent of time spent looking outside valid AoIs: 
0.07312521660312976
Preprocessing participant  27


...

Found 3457 messages...
Block  1
Adding AoIs... 
Epoching... 
Aligning to AoI...
Block  2
Adding AoIs... 
Epoching... 
Aligning to AoI...
Block  3
Adding AoIs... 
Epoching... 
Aligning to AoI...
Block  4
Adding AoIs... 
Epoching... 
Aligning to AoI...
Block  5
Adding AoIs... 
Epoching... 
Aligning to AoI...
Block  6
Adding AoIs... 
Epoching... 
Aligning to AoI...
Block  7
Adding AoIs... 
Epoching... 
Aligning to AoI...
Block  8
Adding AoIs... 
Epoching... 
Aligning to AoI...
Block  9
Adding AoIs... 
Epoching... 
Aligning to AoI...
Block  10
Adding AoIs... 
Epoching... 
Aligning to AoI...
Percent of time spent looking outside valid AoIs: 
0.12411152969289099
Preprocessing participant  70


<ipython-input-17-90a5b70bb05f>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  centers['Block'] = [int(s.replace(sub + 'block' + '_', "")) for s in list(centers['Unnamed: 0'].values)]


Found 3577 messages...
Block  1
Adding AoIs... 
Epoching... 
Aligning to AoI...
Block  2
Adding AoIs... 
Epoching... 
Aligning to AoI...
Block  3
Adding AoIs... 
Epoching... 
Aligning to AoI...
Block  4
Adding AoIs... 
Epoching... 
Aligning to AoI...
Block  5
Adding AoIs... 
Epoching... 
Aligning to AoI...
Block  6
Adding AoIs... 
Epoching... 
Aligning to AoI...
Block  7
Adding AoIs... 
Epoching... 
Aligning to AoI...
Block  8
Adding AoIs... 
Epoching... 
Aligning to AoI...
Block  9
Adding AoIs... 
Epoching... 
Aligning to AoI...
Block  10
Adding AoIs... 
Epoching... 
Aligning to AoI...
Percent of time spent looking outside valid AoIs: 
0.10697581358394494
Preprocessing participant  71


<ipython-input-17-90a5b70bb05f>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  centers['Block'] = [int(s.replace(sub + 'block' + '_', "")) for s in list(centers['Unnamed: 0'].values)]


Found 3474 messages...
Block  1
Adding AoIs... 
Epoching... 
Aligning to AoI...
Block  2
Adding AoIs... 
Epoching... 
Aligning to AoI...
Block  3
Adding AoIs... 
Epoching... 
Aligning to AoI...
Block  4
Adding AoIs... 
Epoching... 
Aligning to AoI...
Block  5
Adding AoIs... 
Epoching... 
Aligning to AoI...
Block  6
Adding AoIs... 
Epoching... 
Aligning to AoI...
Block  7
Adding AoIs... 
Epoching... 
Aligning to AoI...
Block  8
Adding AoIs... 
Epoching... 
Aligning to AoI...
Block  9
Adding AoIs... 
Epoching... 
Aligning to AoI...
Block  10
Adding AoIs... 
Epoching... 
Aligning to AoI...
Percent of time spent looking outside valid AoIs: 
0.05092530759433057
